# Model Implementation for 3D Cell Tracking


In [5]:
!pip install torchsummary 

In [6]:
from torch.utils.data import DataLoader, random_split
from torch.utils.data.sampler import WeightedRandomSampler
from torchvision.datasets import ImageFolder
from torchvision import transforms
import torch
import torch.nn as nn
import numpy as np
import random 
import matplotlib.pyplot as plt
from torchvision import models
from torchsummary import summary

from torchvision import datasets
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler


# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## Designed VGG Model from Exercise 9

We will use a VGG network to classify the synapse images. The input to the network will be a 2D image as provided by your dataloader. The output will be a vector of six floats, corresponding to the probability of the input to belong to the six classes.

Implement a VGG network with the following specificatons:

* the constructor takes the size of the 2D input image as height and width
* the network starts with a downsample path consisting of:
    * one convolutional layer, kernel size (3, 3), to create 12 `fmaps`
    * `nn.BatchNorm2d` over those feature maps
    * `nn.ReLU` activation function
    * `nn.Conv2d` layer, kernel size (3, 3), to create 12 `fmaps`
    * `nn.BatchNorm2d` over those feature maps
    * `nn.ReLU` activation function
    * `nn.MaxPool2d` with a `downsample_factor` of (2, 2) at each level
* followed by three more downsampling paths like the one above, every time doubling the number of `fmaps` (i.e., the second one will have 24, the third 48, and the fourth 96). Make sure to keep track of the `current_fmaps` each time!
* then two times:
    * `nn.Linear` layer with `out_features=4096`. Be careful withe in `in_features` of the first one, which will depend on the size of the previous output!
    * `nn.ReLU` activation function
    * `nn.DropOut`
* Finally, one more fully connected layer with
    * `nn.Linear` to the 6 classes
    * no activation function 

Original One (from https://blog.paperspace.com/vgg-from-scratch-pytorch/)
https://github.com/pytorch/vision/blob/6db1569c89094cf23f3bc41f79275c45e9fcb3f3/torchvision/models/vgg.py#L24

# Create and Load Artificial Data
Make a fake dataset to test on the VGG model while waiting for data.


In [33]:
#original data size = 512x712x34 

#fd_class1 = np.random.randn(1, 1, 128,128,128) + 0.5
#fd_class2 = np.random.randn(1,1,128,128,128)
fd_class1 = np.random.randn(1, 1, 128,128,5) + 0.5
fd_class2 = np.random.randn(1,1,128,128,5)
y1 = 1
y2 = 0
# fd_class1 = np.expand_dims(fd_class1, axis=0)
# fd_class2 = np.expand_dims(fd_class2, axis=0)

loader = [(fd_class1, y1), (fd_class2,y2)]

#Split
# train_set_size = int(len(fd_class1) * 0.7)
# valid_set_size = int(len(fd_class1) * 0.2)
# test_data_size = len(fd_class1) - train_set_size - valid_set_size
    
# train_data_C1, val_data_C1, test_data_C1 = random_split(
#     fd_class1,
#     [train_set_size, valid_set_size, test_data_size],
#     generator=torch.Generator().manual_seed(23061912))

# train_data_C2, val_data_C2, test_data_C2 = random_split(
#     fd_class2,
#     [train_set_size, valid_set_size, test_data_size],
#     generator=torch.Generator().manual_seed(23061912))

#train = np.ndarray.flatten(fd_class1)
#train2 = np.ndarray.flatten(fd_class2)




In [8]:
#sampler = balanced_sampler(train_data_C1)
#dataloader = DataLoader(train_data_C1, batch_size=8, drop_last=True)

# Define the Model

In [9]:
class Vgg3D(torch.nn.Module):

    def __init__(self, input_size, output_classes, downsample_factors, fmaps=12):

        super(Vgg3D, self).__init__()

        self.input_size = input_size
        self.downsample_factors = downsample_factors
        self.output_classes = 2

        current_fmaps, h, w, d = tuple(input_size)
        current_size = (h, w,d)

        features = []
        for i in range(len(downsample_factors)):

            features += [
                torch.nn.Conv3d(current_fmaps,fmaps,kernel_size=3,padding=1),
                torch.nn.BatchNorm3d(fmaps),
                torch.nn.ReLU(inplace=True),
                torch.nn.Conv3d(fmaps,fmaps,kernel_size=3,padding=1),
                torch.nn.BatchNorm3d(fmaps),
                torch.nn.ReLU(inplace=True),
                torch.nn.MaxPool3d(downsample_factors[i])
            ]

            current_fmaps = fmaps
            fmaps *= 2

            size = tuple(
                int(c/d)
                for c, d in zip(current_size, downsample_factors[i]))
            check = (
                s*d == c
                for s, d, c in zip(size, downsample_factors[i], current_size))
            assert all(check), \
                "Can not downsample %s by chosen downsample factor" % \
                (current_size,)
            current_size = size

        self.features = torch.nn.Sequential(*features)

        classifier = [
            torch.nn.Linear(current_size[0] *current_size[1]*current_size[2] *current_fmaps,4096),
            torch.nn.ReLU(inplace=True),
            torch.nn.Dropout(),
            torch.nn.Linear(4096,4096),
            torch.nn.ReLU(inplace=True),
            torch.nn.Dropout(),
            torch.nn.Linear(4096,output_classes)
        ]

        self.classifier = torch.nn.Sequential(*classifier)
    
    def forward(self, raw):

        # add a channel dimension to raw
        # shape = tuple(raw.shape)
        # raw = raw.reshape(shape[0], 1, shape[1], shape[2])
        
        # compute features
        f = self.features(raw)
        f = f.view(f.size(0), -1)
        
        # classify
        y = self.classifier(f)

        return y

# Training and Evaluation

# Loss Functions

We'll probably need to test some different loss functions. List some here:
Contrastive loss
cosine similarity
triplet loss



In [10]:
class ContrastiveLoss(nn.Module):
    "Contrastive loss function"

    def __init__(self, margin=2.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, output1, output2, label):
        euclidean_distance = F.pairwise_distance(output1, output2)
        loss_contrastive = torch.mean(
            (1 - label) * torch.pow(euclidean_distance, 2)
            + (label)
            * torch.pow(torch.clamp(self.margin - euclidean_distance, min=0.0), 2)
        )

        return loss_contrastive

In [36]:
input_size = (1, 64, 64, 2)
downsample_factors =[(2, 2, 1), (2, 2, 1), (2, 2, 1), (2, 2, 1)];
output_classes = 12

# create the model to train
model = Vgg3D(input_size, output_classes,  downsample_factors = downsample_factors)
model = model.to(device)

summary(model, input_size)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv3d-1        [-1, 12, 64, 64, 2]             336
       BatchNorm3d-2        [-1, 12, 64, 64, 2]              24
              ReLU-3        [-1, 12, 64, 64, 2]               0
            Conv3d-4        [-1, 12, 64, 64, 2]           3,900
       BatchNorm3d-5        [-1, 12, 64, 64, 2]              24
              ReLU-6        [-1, 12, 64, 64, 2]               0
         MaxPool3d-7        [-1, 12, 32, 32, 2]               0
            Conv3d-8        [-1, 24, 32, 32, 2]           7,800
       BatchNorm3d-9        [-1, 24, 32, 32, 2]              48
             ReLU-10        [-1, 24, 32, 32, 2]               0
           Conv3d-11        [-1, 24, 32, 32, 2]          15,576
      BatchNorm3d-12        [-1, 24, 32, 32, 2]              48
             ReLU-13        [-1, 24, 32, 32, 2]               0
        MaxPool3d-14        [-1, 24, 16

In [12]:
#Training length
epochs = 2000

#loss_function = torch.nn.BCELoss()
loss_function = torch.nn.CosineEmbeddingLoss()
#loss_function = ContrastiveLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=0.0005)

# Training Test

In [52]:
from tqdm import tqdm

def train():
    if torch.cuda.is_available():
        device = torch.device("cuda")
    else:
        device = torch.device("cpu")
        
    loss=[] 
    counter=[]
    for epoch in tqdm(range(epochs)):
        epoch_loss = 0
        for i, (x, y) in enumerate(loader):
            
            x = torch.from_numpy(x).to(device).float()
            y = torch.from_numpy(np.array([y])).to(device).float()
            
            #vol0, vol1 , label = data
            #vol0, vol1 , label = vol0.to(device), vol1.to(device) , label.to(device)
            
            optimizer.zero_grad()
            
            pred = model(x).mean(axis = 1).sigmoid()
            #print(model(x))
            #output1 = model(vol0)
            #output2 = model(vol1)
            
            #print(pred,y)
            loss = loss_function(pred, y)
            #loss_contrastive = loss_function(output1,output2,label)
            
            loss.backward()
            optimizer.step()    
            epoch_loss += loss
            
        print(f"epoch {epoch}, training loss={epoch_loss}")
    
    #show_plot(epoch, loss)
    
    return model

model = train()

  0%|                                        | 1/2000 [00:02<1:21:15,  2.44s/it]

epoch 0, training loss=3.0052499771118164


  0%|                                        | 2/2000 [00:04<1:18:54,  2.37s/it]

epoch 1, training loss=3.012202262878418


  0%|                                        | 3/2000 [00:07<1:18:07,  2.35s/it]

...

epoch 154, training loss=9.298368240706623e-06


  8%|██▉                                   | 156/2000 [06:03<1:11:35,  2.33s/it]

epoch 155, training loss=0.0


  8%|██▉                                   | 156/2000 [06:05<1:12:05,  2.35s/it]


KeyboardInterrupt: 

# Implementing the Siamese Network

The above training is just to test if the VGG model works for 3D data. Here, the training will take two pairs of images and calculate the loss from both pairs of images.

In [13]:
from tqdm import tqdm

def train():
    if torch.cuda.is_available():
        device = torch.device("cuda")
    else:
        device = torch.device("cpu")
        
    loss=[] 
    counter=[]
    for epoch in tqdm(range(epochs)):
        epoch_loss = 0
        for i, (x, y) in enumerate(loader):
            
            x = torch.from_numpy(x).to(device).float()
            y = torch.from_numpy(np.array([y])).to(device).float()
            
            vol0, vol1 , label0, label1 = data
            vol0, vol1 , label = vol0.to(device), vol1.to(device), label.to(device)
            
            optimizer.zero_grad()
            
            output1 = model(vol0)
            output2 = model(vol1)
            
            if(label0 == label1):
                isMatch = 1;
            else:
                isMatch = 0;
                        
            loss = loss_function((output1, output2), isMatch)
            
            loss.backward()
            optimizer.step()    
            epoch_loss += loss
            
        print(f"epoch {epoch}, training loss={epoch_loss}")
    
    return model

def validate():    
    model.eval()
    #dataloader = DataLoader(validation_dataset, batch_size=32)
    
    return evaluate(dataloader, 'validate')

def test():    
    model.eval()
    dataloader = DataLoader(test_dataset, batch_size=32)
    
    return evaluate(dataloader, 'test')

model = train()

  0%|                                                  | 0/2000 [00:00<?, ?it/s]


NameError: name 'data' is not defined

In [32]:
for i, (x, y) in enumerate(loader):
    #print(i)
    #print(np.shape(x))
    print(y)
#print(x)
#print(y)


1
0
